In [6]:
import torchvision.transforms as transforms
import torch

from src.concept_bottleneck.dataset import CUB200_2011

# According to: https://pytorch.org/hub/pytorch_vision_inception_v3/
preprocess = transforms.Compose(
    [
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ]
)

training_data: CUB200_2011[torch.Tensor] = CUB200_2011(train=True, transform=preprocess)
test_data: CUB200_2011[torch.Tensor] = CUB200_2011(train=False, transform=preprocess)


Using downloaded and verified file: /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data/CUB_200_2011.tgz
Extracting /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data/CUB_200_2011.tgz to /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data
Using downloaded and verified file: /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data/CUB_200_2011.tgz
Extracting /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data/CUB_200_2011.tgz to /home/shuangwu/interactive-concept-bottleneck/src/concept_bottleneck/data


In [9]:
from torch.utils.data import DataLoader

batch_size = 8
num_workers = 2

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=2)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model: torch.nn.Module = torch.hub.load(
    "pytorch/vision:v0.10.0",
    "inception_v3",
    init_weights=False,
    num_classes=training_data.num_attributes,
)
model = model.to(device)
print(model)


Using cuda device


Using cache found in /home/shuangwu/.cache/torch/hub/pytorch_vision_v0.10.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri